# 关联分析 - Apriori

所谓关联分析（关联规则学习）指的是从大规模数据集中学习到物品之间隐含的关系，Apriori基于该原理实现；

In [2]:
from numpy import *

import matplotlib.pyplot as plt

%matplotlib inline

dataPath = '../../../git_mlaction/machinelearninginaction/Ch11/'

## 关联分析

Apriori算法：
* 优点：易编码实现；
* 缺点：在大数据集上速度慢；
* 适用数据类型：数值型或标称型数据；

关联分析是从大量数据中提取物品的隐含关系，这种关系有两种类型：
* 频繁项集：指经常出现在一起的物品的集合；
* 关联规则：指两种物品之间存在很强的关系；

两个指标：
* 支持度：一个项集的支持度被定义为数据集中包含该项集的比例值，最大为1，最小为0，该值可用于类似预剪枝的作用，提前终止算法；
* 可信度：可信度用于评估关联规则，例如对于{尿布}->{啤酒}这样一对关联规则，它的可信度等于(支持度{尿布,啤酒})/(支持度{尿布})，也就是在所有包含尿布的记录中，我们的规则占比是多少，同样从0到1；

从上述指标可以看出，如果要获取到所有支持度大于0.5的项集，可以穷举所有组合，然后统计其出现次数，除以总数即可，但是当物品很多时，这个组合（排列组合问题）也是很多的，如何优化这部分算法的性能，就是Apriori主要做的工作；

## Apriori原理

对于物品之间的各种组合关系，可以通过下图来清晰呈现，假设我们只有四种物品，分别是0,1,2,3，此时所有组合如下：

![物品排列组合](image/apriori.png)

可以看到，所有组合总数为2\*\*N - 1，当物品种类为4时，有15种组合，那么当物品种类为100时呢，很快就超出了普通计算机的承受能力；

Apriori原理简单描述是这样的：当一个项集频繁（高支持度）时，它的所有子集也是高频繁的，反过来就是当一个项目非频繁时，它的所有超集也都是非频繁的，即如果我们随机证明了{2,3}是非频繁的，那么我们就无须证明也能确定{0,2,3},{1,2,3},{0,1,2,3}同样是非频繁的，这就避免了由于种类的增加，导致要遍历的此处指数级上升的问题；

## 使用Apriori算法发现频繁项集

算法基本流程：
1. 从单个项集开始，计算各个项集是否满足最小支持度；
2. 遍历所有满足的项集，遍历其各个组合；
3. 重复1,2直到没有项集能够计算为止；

对于最小支持度的设置要求较高，太低了会导致算法运行速度过慢，太高了会导致得到很少的频繁项集，看实际需求而定；

### 生成候选项集

In [3]:
def loadTestDataSet():
    return [[1,3,4], [2,3,5], [1,2,3,5], [2,5]]

In [12]:
def createC1(dataSet):
    C1 = []
    for item in dataSet:
        for v in item:
            if not [v] in C1:
                C1.append([v])
    C1.sort()
    return map(frozenset, C1) # 创建不变集合，注意frozenset对列表元素的要求

createC1(loadTestDataSet())

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [13]:
# 该函数负责从数据集D中计算出Ck中满足最小支持度的集合L
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for data in D: # 遍历所有数据，统计Ck中各个项集的出现次数
        for c in Ck:
            if c.issubset(data):
                if ssCnt.has_key(c):
                    ssCnt[c] += 1
                else:
                    ssCnt[c] = 1
    numD = float(len(D)) # 数据总数
    L = []
    supportData = {}
    for k in ssCnt.keys():
        support = ssCnt[k]/numD
        if support >= minSupport:
            L.insert(0, k)
        supportData[k] = support
    return L, supportData

scanD(loadTestDataSet(), createC1(loadTestDataSet()), 0.5)

([frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
 {frozenset({4}): 0.25,
  frozenset({5}): 0.75,
  frozenset({2}): 0.75,
  frozenset({3}): 0.75,
  frozenset({1}): 0.5})

### 组织完整的Apriori算法

In [21]:
# 定义函数实现由L1到C2的过程，即排列组合各个项集到一个更多元素的层次
def aprioriGen(Lk, k):
    Ck = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            # 只合并前k-2个项相同的项集，避免出现长度超过k的项集，比如{0,1}和{0,2}合并，满足条件，结果为{0,1,2}
            # 但是如果是{0,1}和{2,3}则不满足前k-1个项相同的条件，因此不会合并出{0,1,2,3}这种case
            L1, L2 = list(Lk[i])[:k-2], list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1 == L2:
                Ck.append(Lk[i] | Lk[j])
    return Ck

gen1_2 = aprioriGen(scanD(loadTestDataSet(), createC1(loadTestDataSet()), 0.5)[0], 2)
print gen1_2
print aprioriGen(gen1_2, 3)

[frozenset([1, 3]), frozenset([1, 2]), frozenset([1, 5]), frozenset([2, 3]), frozenset([3, 5]), frozenset([2, 5])]
[frozenset([1, 2, 3]), frozenset([1, 3, 5]), frozenset([1, 2, 5]), frozenset([2, 3, 5])]


In [32]:
# Apriori算法主流程
def apriori(dataSet, minSupport=0.5):
    C1 = createC1(dataSet)
    D = map(set, dataSet)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while(len(L[k-2])>1): # 其实某一层的项集数为1或更小的时候就结束了，因为1个项集无法组合成更高维度的超集
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [33]:
apriori(loadTestDataSet())

([[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
  [frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3}), frozenset({3, 5})],
  [frozenset({2, 3, 5})]],
 {frozenset({5}): 0.75,
  frozenset({3}): 0.75,
  frozenset({2, 3, 5}): 0.5,
  frozenset({1, 2}): 0.25,
  frozenset({1, 5}): 0.25,
  frozenset({3, 5}): 0.5,
  frozenset({4}): 0.25,
  frozenset({2, 3}): 0.5,
  frozenset({2, 5}): 0.75,
  frozenset({1}): 0.5,
  frozenset({1, 3}): 0.5,
  frozenset({2}): 0.75})

In [34]:
apriori(loadTestDataSet(), 0.7)

([[frozenset({3}), frozenset({2}), frozenset({5})], [frozenset({2, 5})]],
 {frozenset({5}): 0.75,
  frozenset({3}): 0.75,
  frozenset({3, 5}): 0.5,
  frozenset({4}): 0.25,
  frozenset({2, 3}): 0.5,
  frozenset({2, 5}): 0.75,
  frozenset({1}): 0.5,
  frozenset({2}): 0.75})

可以看到，在apriori算法中，我们将最小支持度从0.5提高到0.75，符合条件的组合个数从9个下降到4个；

## 从频繁项集中挖掘关联规则

关联规则的发现是依赖于频繁项集的（从计算公式中也可以看出，可信度是依赖于支持度的），基本算法流程类似频繁项集的获取，看下图：

![关联规则计算](image/rules.png)

与频繁项集不同的只是在项集前出现一个xxx->用于表示该项集潜在的关联规则；

In [36]:
# 定义函数计算可信度
def calcConf(freqSet, H, supportData, br1, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print freqSet - conseq, '---->', conseq, ' conf:', conf
            br1.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [38]:
# 定义函数生成后选规则集合
def rulesFromConseq(freqSet, H, supportData, br1, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m+1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, br1, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, br1, minConf)

In [40]:
# 关联规则主函数
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)): # 注意此处从1开始
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i>1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

In [42]:
L, supportData = apriori(loadTestDataSet(), minSupport=0.5)
rules = generateRules(L, supportData)
print rules

frozenset([1]) ----> frozenset([3])  conf: 1.0
frozenset([5]) ----> frozenset([2])  conf: 1.0
frozenset([2]) ----> frozenset([5])  conf: 1.0
[(frozenset([1]), frozenset([3]), 1.0), (frozenset([5]), frozenset([2]), 1.0), (frozenset([2]), frozenset([5]), 1.0)]


In [43]:
rules = generateRules(L, supportData, 0.5)
print rules

frozenset([3]) ----> frozenset([1])  conf: 0.666666666667
frozenset([1]) ----> frozenset([3])  conf: 1.0
frozenset([5]) ----> frozenset([2])  conf: 1.0
frozenset([2]) ----> frozenset([5])  conf: 1.0
frozenset([3]) ----> frozenset([2])  conf: 0.666666666667
frozenset([2]) ----> frozenset([3])  conf: 0.666666666667
frozenset([5]) ----> frozenset([3])  conf: 0.666666666667
frozenset([3]) ----> frozenset([5])  conf: 0.666666666667
frozenset([5]) ----> frozenset([2, 3])  conf: 0.666666666667
frozenset([3]) ----> frozenset([2, 5])  conf: 0.666666666667
frozenset([2]) ----> frozenset([3, 5])  conf: 0.666666666667
[(frozenset([3]), frozenset([1]), 0.6666666666666666), (frozenset([1]), frozenset([3]), 1.0), (frozenset([5]), frozenset([2]), 1.0), (frozenset([2]), frozenset([5]), 1.0), (frozenset([3]), frozenset([2]), 0.6666666666666666), (frozenset([2]), frozenset([3]), 0.6666666666666666), (frozenset([5]), frozenset([3]), 0.6666666666666666), (frozenset([3]), frozenset([5]), 0.6666666666666666)

可以看到，降低可信度后，生成的关联规则明显多于降低前，也符合算法思想；

## 示例：发现国会投票中的模式

## 示例：发现毒蘑菇的相似特征

In [45]:
mushDatSet = [line.split() for line in open(dataPath+'mushroom.dat').readlines()]
mushDatSet[0]

['1',
 '3',
 '9',
 '13',
 '23',
 '25',
 '34',
 '36',
 '38',
 '40',
 '52',
 '54',
 '59',
 '63',
 '67',
 '76',
 '85',
 '86',
 '90',
 '93',
 '98',
 '107',
 '113']

In [46]:
# 在毒蘑菇数据上运行apriori算法
L, suppData = apriori(mushDatSet, minSupport=0.3)
# 在结果中搜索包含有有毒特征值的项集组合
for item in L[1]: # 暂时只从只有一个元素的项集中查找，否则结果太多
    if item.intersection('2'):
        print item

frozenset(['2', '59'])
frozenset(['39', '2'])
frozenset(['2', '67'])
frozenset(['2', '34'])
frozenset(['2', '23'])
frozenset(['2', '86'])
frozenset(['76', '2'])
frozenset(['90', '2'])
frozenset(['2', '53'])
frozenset(['93', '2'])
frozenset(['63', '2'])
frozenset(['2', '28'])
frozenset(['2', '85'])
frozenset(['2', '36'])


## 小结

关联分析是发现大数据集中隐含关系的一种方式，包括频繁项集和关联规则，二者对应指标为支持度和可信度，由于关联分析需要穷举所有可能的物品种类组合，因此非常耗时，此时就需要一种更加智能的算法来优化，Apriori算法就是用于此目的，它的思想很简单：如果一个项集是频繁的，那么它的所有子集都是频繁的，反之，如果一个项集是不频繁的，那么它所有的超集都是不频繁的，借助这一算法，我们可以将组合数大大降低；